## *COVID-19 IMAGE CLASSIFICATION*

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

In [ ]:
import zipfile

local_file = '/content/covid19-radiography-database.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import os
from os import makedirs

base_ref = "/tmp/" # base directory where the data set will be stored
sub_dirs = ["/train", "/validate"] 
for sub_dir in sub_dirs:
  label_dirs = ["/covid", "/normal", "/viral_pneumonia"]
  for label_dir in label_dirs:
    new_dir = base_ref + sub_dir + label_dir
    os.makedirs(new_dir, exist_ok=True)

In [ ]:
from os import listdir
from random import random
from shutil import copyfile

src_directory = '/content/COVID-19_Radiography_Dataset/COVID'
val_ratio = 0.25

for file in listdir(src_directory):
  src = src_directory + '/' + file
  dst_dir = 'train/'
  if random() < val_ratio:
    dst_dir = 'validate/'
  dst = base_ref + dst_dir + 'covid/' + file
  copyfile(src, dst)

In [ ]:
src_directory = '/content/COVID-19_Radiography_Dataset/Normal'

for file in listdir(src_directory):
  src = src_directory + '/' + file
  dst_dir = 'train/'
  if random() < val_ratio:
    dst_dir = 'validate/'
  dst = base_ref + dst_dir + 'normal/' + file
  copyfile(src, dst)

In [ ]:
src_directory = '/content/COVID-19_Radiography_Dataset/Viral Pneumonia'

for file in listdir(src_directory):
  src = src_directory + '/' + file
  dst_dir = 'train/'
  if random() < val_ratio:
    dst_dir = 'validate/'
  dst = base_ref + dst_dir + 'viral_pneumonia/' + file
  copyfile(src, dst)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

test_datagen = ImageDataGenerator(
    rescale = 1./255.
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/tmp/train',
    batch_size = 64,
    target_size = (299, 299),
    class_mode = 'sparse',
    subset='training',

)

validation_generator = test_datagen.flow_from_directory(
    '/tmp/validate',
    batch_size = 64,
    target_size = (299, 299),
    class_mode = 'sparse',
    subset = 'validation'
)

In [ ]:
import os

print(train_generator.class_indices)
label = '\n'.join(sorted(train_generator.class_indices))
with open('label.txt', mode='w') as f:
  f.write(label)

In [ ]:
from tensorflow.keras.applications import InceptionV3

img_shape = (299, 299, 3)
base_model = InceptionV3(
    input_shape = img_shape,
    include_top = False
)

for layer in base_model.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import layers

x = layers.Flatten()(base_model.layers[-1].output)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.inputs, outputs = x)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
)

In [ ]:
history = model.fit(
    train_generator, 
    epochs = 20, 
    validation_data = validation_generator,
    callbacks = [early_stopping]
)

In [ ]:
model.save('model.h5')
files.download('/content/model.h5')

In [ ]:
uploaded = files.upload()

In [ ]:
label = {
    0 : 'Covid',
    1 : 'Normal',
    2 : 'Viral Pneumonia'
}

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from keras.applications.inception_v3 import preprocess_input

img = load_img('/content/photo_5.jpg', target_size=(299,299))
img = img_to_array(img)
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
img = preprocess_input(img)

y_hat = model.predict(img)
print(y_hat)
ans = label[np.argmax(y_hat)]
print(ans)